In [72]:
import geopandas as gpd

met_counties = gpd.read_file('../data/boundaries/Metropolitan_Counties_December_2018_FEB_in_England_2022_-7757264906552428134.gpkg')
centroids = gpd.read_file('../data/boundaries/MSOA_Dec_2011_PWC_in_England_and_Wales_2022_8663885760129340094.gpkg')

mcr = met_counties[met_counties['mcty18nm'] == 'Greater Manchester']

# Only centroids within Greater Manchester
centroids = centroids[centroids.within(mcr.geometry.iloc[0])]

# Convert CRS to WS84
centroids = centroids.to_crs('EPSG:4326')
centroids = centroids.reset_index(drop=True)
centroids = centroids.iloc[:24,]

centroids_coords = {code: (lat, lon) for code, lat, lon in zip(centroids['msoa11cd'], centroids.geometry.y, centroids.geometry.x)}
centroids_coords

{'E02001236': (53.489371101297365, -2.0599674295183945),
 'E02001237': (53.48807166280751, -2.038941532097706),
 'E02001234': (53.49816689893988, -2.090822925267231),
 'E02001235': (53.489596462049995, -2.079096289183832),
 'E02001232': (53.49851112950124, -2.0744940637778924),
 'E02001233': (53.49809192423287, -2.1077100499563555),
 'E02001230': (53.506475834573244, -2.090231217284018),
 'E02001231': (53.503133191470226, -2.028562009451858),
 'E02001238': (53.48577037509783, -2.135919519965099),
 'E02001239': (53.48522804546479, -2.1505514032693487),
 'E02001106': (53.561034662937, -2.1162609634578384),
 'E02001187': (53.44504978844013, -2.154513155611838),
 'E02001186': (53.429748432552714, -2.4387569542408714),
 'E02001107': (53.55326073401111, -2.1440491744446106),
 'E02001185': (53.45144631787275, -2.4154486625535676),
 'E02001104': (53.56510745304541, -2.0753883364767636),
 'E02001184': (53.47373363087366, -2.279739136744257),
 'E02001105': (53.5623806456716, -2.131013514318461),

In [73]:
import pandas as pd
import asyncio
import aiohttp
import nest_asyncio

nest_asyncio.apply()

async def get_isochrone_async(session: aiohttp.ClientSession, lat: float, lon: float, duration: int, arrival_time: str) -> gpd.GeoDataFrame:
    """Get isochrone from the isochrone API.
    Args: 
        lat: Latitude of the origin point
        lon: Longitude of the origin point
        duration: Duration of the isochrone in seconds
    """

    request = f"http://localhost:8000/isochrone?lat={lat}&lon={lon}&arrival_time={arrival_time}&duration={duration}"
    async with session.get(request) as response:
        response.raise_for_status()
        geojson = await response.text()
        gdf = gpd.read_file(geojson)
        return gdf

async def get_isochrones_async(station_coords: dict, duration: int, arrival_time: str) -> gpd.GeoDataFrame:
    """Get isochrones for a list of stations asynchronously.
    Args:
        station_coords: Dictionary of station names and coordinates
        duration: Duration of the isochrones in seconds
    """
    async with aiohttp.ClientSession() as session:
        tasks = [get_isochrone_async(session, lat, lon, duration, arrival_time) for lat, lon in station_coords.values()]
        isochrones = await asyncio.gather(*tasks)
        isochrones = pd.concat(isochrones, ignore_index=True)
        isochrones['station'] = station_coords.keys()
        return isochrones


def get_plot():
    isochrones = asyncio.run(get_isochrones_async(centroids_coords, 1800, "08:30:00"))
    plot = isochrones.explore(column='station')
    return plot

plot = get_plot()
plot